# Hello Node Kubernetes

**Learning Objectives**
 * Create a Node.js server
 * Create a Docker container image
 * Create a container cluster and a Kubernetes pod
 * Scale up your services

## Overview

The goal of this hands-on lab is for you to turn code that you have developed into a replicated application running on Kubernetes, which is running on Kubernetes Engine. For this lab the code will be a simple Hello World node.js app.

Here's a diagram of the various parts in play in this lab, to help you understand how the pieces fit together with one another. Use this as a reference as you progress through the lab; it should all make sense by the time you get to the end (but feel free to ignore this for now).

<img src='../assets/k8s_hellonode_overview.png' width="500"/>

## Create a Node.js server

The file `./src/server.js` contains a simple Node.js server. Use `cat` to examine the contents of that file.

In [2]:
!cat ./src/server.js

const http = require('http');

const hostname = '0.0.0.0';
const port = 80;

const server = http.createServer((req, res) => {
    res.statusCode = 200;
      res.setHeader('Content-Type', 'text/plain');
        res.end('No hello for no-world!\n');
});

server.listen(port, hostname, () => {
    console.log('Server running at http://%s:%s/', hostname, port);
});

process.on('SIGINT', function() {
    console.log('Caught interrupt signal and will exit');
    process.exit();
});


Start the server by running `node server.js` in the cell below. Open a terminal and type
```bash
curl http://localhost:8000
```
to see what the server outputs. 

In [6]:
!node ./src/server.js

Server running at http://0.0.0.0:8000/
^C
Caught interrupt signal and will exit


You should see the output `"Hello World!"`. Once you've verfied this, interupt the above running cell by hitting the stop button.

## Create and build a Docker image

Now we will create a docker image called `hello_node.docker` that will do the following:

 1. Start from the node image found on the Docker hub by inhereting from `node:6.9.2`
 2. Expose port 8000
 3. Copy the `./src/server.js` file to the image
 4. Start the node server as we previously did manually using `CMD`

**Exercise**

Edit the file called called `hello_node.docker` in the `dockerfiles` folder to complete all the TODOs.

Next, build the image in your project using `docker build`.

In [7]:
import os

PROJECT_ID = "qwiklabs-gcp-01-4098456db096" # REPLACE WITH YOUR PROJECT NAME
os.environ["PROJECT_ID"] = PROJECT_ID

**Exercise**

Use `docker build` to build the image from dockerfile you created above. Tag the image as `gcr.io/[project-id]/hello-node:v1`.

In [11]:
%%bash
docker build -f dockerfiles/hello_node.docker -t gcr.io/${PROJECT_ID}/hello-node:v1 .

Sending build context to Docker daemon  187.9kB
Step 1/4 : FROM node:6.9.2
 ---> faaadb4aaf9b
Step 2/4 : EXPOSE 8000
 ---> Running in 0343a08c0457
Removing intermediate container 0343a08c0457
 ---> ca24bc9011c0
Step 3/4 : COPY ./src/server.js .
 ---> 172b568f82e5
Step 4/4 : CMD ["node", "server.js"]
 ---> Running in 8b7318d8f8ef
Removing intermediate container 8b7318d8f8ef
 ---> dbae68f59f7b
Successfully built dbae68f59f7b
Successfully tagged gcr.io/qwiklabs-gcp-01-4098456db096/hello-node:v1


It'll take some time to download and extract everything, but you can see the progress bars as the image builds. Once complete, test the image locally by running a Docker container as a daemon on port 8000 from your newly-created container image.

**Exercise**

Run the container using `docker run` on the port 8000.

In [12]:
!docker images

REPOSITORY                                       TAG       IMAGE ID       CREATED         SIZE
gcr.io/qwiklabs-gcp-01-4098456db096/hello-node   v1        dbae68f59f7b   3 minutes ago   655MB
<none>                                           <none>    3590e8bef9be   4 minutes ago   655MB
gcr.io/qwiklabs-gcp-01-4098456db096/node-app     0.2       037fd15d4996   23 hours ago    884MB
gcr.io/inverting-proxy/agent                     <none>    fe507176d0e6   2 months ago    1.73GB
node                                             6.9.2     faaadb4aaf9b   4 years ago     655MB


In [13]:
%%bash
docker run -p 8000:8000 -d gcr.io/${PROJECT_ID}/hello-node:v1

68b6d2665007e6c04bc0572294c8e3598655dd5915df88524617858e6cd7e607


Your output should look something like this:
```bash
b16e5ccb74dc39b0b43a5b20df1c22ff8b41f64a43aef15e12cc9ac3b3f47cfd
```

Right now, since you used the `--d` flag, the container process is running in the background. You can verify it's running using `curl` as before.

In [14]:
!curl http://localhost:8000

No hello for no-world!
from Val


Now, stop running the container. Get the container id using `docker ps` and then terminate using `docker stop`

In [19]:
!docker ps

CONTAINER ID   IMAGE                                              COMMAND                  CREATED        STATUS        PORTS                  NAMES
e0d82dcb1cd9   gcr.io/qwiklabs-gcp-01-4098456db096/node-app:0.2   "node ./src/app.js"      23 hours ago   Up 23 hours   0.0.0.0:4000->80/tcp   compassionate_turing
facda91186f6   gcr.io/inverting-proxy/agent                       "/bin/sh -c '/opt/bi…"   24 hours ago   Up 24 hours                          proxy-agent


In [16]:
# your container id will be different
!docker stop 68b6d2665007

68b6d2665007


Now that the image is working as intended, push it to the Google Container Registry, a private repository for your Docker images, accessible from your Google Cloud projects. First, configure docker uisng your local config file. The initial push may take a few minutes to complete. You'll see the progress bars as it builds.

In [17]:
!gcloud auth configure-docker


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud"
  }
}
Adding credentials for all GCR repositories.
gcloud credential helpers already registered correctly.


**Exercise**

Push the `hello-node:v1` image to your Cloud gcr.

In [18]:
%%bash
docker push gcr.io/qwiklabs-gcp-01-4098456db096/hello-node:v1

The push refers to repository [gcr.io/qwiklabs-gcp-01-4098456db096/hello-node]
c0e0960e8aee: Preparing
381c97ba7dc3: Preparing
604c78617f34: Preparing
fa18e5ffd316: Preparing
0a5e2b2ddeaa: Preparing
53c779688d06: Preparing
60a0858edcd5: Preparing
b6ca02dfe5e6: Preparing
60a0858edcd5: Waiting
53c779688d06: Waiting
b6ca02dfe5e6: Waiting
fa18e5ffd316: Pushed
c0e0960e8aee: Pushed
604c78617f34: Pushed
b6ca02dfe5e6: Layer already exists
381c97ba7dc3: Pushed
60a0858edcd5: Pushed
53c779688d06: Pushed
0a5e2b2ddeaa: Pushed
v1: digest: sha256:24bcf6fd1b9a3595efeefa8ec2ff77f495de1f29ae3f5c7dcf6aceaeaa5cf94c size: 2002


The container image will be listed in your Console. Select `Navigation` > `Container Registry`.

## Create a cluster on GKE

Now you're ready to create your Kubernetes Engine cluster. A cluster consists of a Kubernetes master API server hosted by Google and a set of worker nodes. The worker nodes are Compute Engine virtual machines.

Create a cluster with two n1-standard-1 nodes (this will take a few minutes to complete). You can safely ignore warnings that come up when the cluster builds.

**Note**: You can also create this cluster through the Console by opening the Navigation menu and selecting `Kubernetes Engine` > `Kubernetes clusters` > `Create cluster`.

**Exercise** 

Create a GKE cluster in your project using `gcloud container clusters create`. Your cluster should
 1. be named `hello-world`
 2. have two nodes
 3. consist of `n1-standard-1` machines

In [21]:
%%bash
gcloud container clusters create hello-world --num-nodes 2 --machine-type n1-standard-1 --zone us-central1-a

NAME         LOCATION       MASTER_VERSION   MASTER_IP      MACHINE_TYPE   NODE_VERSION     NUM_NODES  STATUS
hello-world  us-central1-a  1.18.16-gke.502  35.222.74.227  n1-standard-1  1.18.16-gke.502  2          RUNNING


Creating cluster hello-world in us-central1-a...
..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................done.
Created [https://container.googleapis.com/v1/projects/qwiklabs-gcp-01-4098456db096/zones/us-central1-a/clusters/hello-world].
To inspect the contents of your cluster, go to: https://console.c

## Create a pod

A Kubernetes pod is a group of containers tied together for administration and networking purposes. It can contain single or multiple containers. Here you'll use one container built with your `Node.js` image stored in your private container registry. It will serve content on port 8000.

**Exercise**

Create a pod with the `kubectl create` command. You should 
 1. name your deployment `hello-node`
 2. be based on the image you created above called `gcr.io/[project-id]/hello-node:v1

In [22]:
%%bash
kubectl create deployment hello-node --image=gcr.io/${PROJECT_ID}/hello-node:v1

deployment.apps/hello-node created


As you can see, you've created a deployment object. Deployments are the recommended way to create and scale pods. Here, a new deployment manages a single pod replica running the `hello-node:v1` image.

View the deployment using `kubectl get`

In [27]:
!kubectl get deployments

NAME         READY   UP-TO-DATE   AVAILABLE   AGE
hello-node   1/1     1            1           2m34s


Similarly, view the pods created by the deployment by also using `kubectl get`

In [28]:
!kubectl get pods

NAME                         READY   STATUS    RESTARTS   AGE
hello-node-ff9ccfb86-gkrwh   1/1     Running   0          2m58s


Here are some other good kubectl commands you should know. They won't change the state of the cluster. Others can be found [here](https://kubernetes.io/docs/reference/kubectl/overview/).
 * `kubectl cluster-info`
 * `kubectl config view`
 * `kubectl get events`
 * `kubectl logs <pod-name>`

In [25]:
!kubectl cluster-info

Kubernetes control plane is running at https://35.222.74.227
GLBCDefaultBackend is running at https://35.222.74.227/api/v1/namespaces/kube-system/services/default-http-backend:http/proxy
KubeDNS is running at https://35.222.74.227/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy
Metrics-server is running at https://35.222.74.227/api/v1/namespaces/kube-system/services/https:metrics-server:/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [26]:
!kubectl config view

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: DATA+OMITTED
    server: https://35.222.74.227
  name: gke_qwiklabs-gcp-01-4098456db096_us-central1-a_hello-world
contexts:
- context:
    cluster: gke_qwiklabs-gcp-01-4098456db096_us-central1-a_hello-world
    user: gke_qwiklabs-gcp-01-4098456db096_us-central1-a_hello-world
  name: gke_qwiklabs-gcp-01-4098456db096_us-central1-a_hello-world
current-context: gke_qwiklabs-gcp-01-4098456db096_us-central1-a_hello-world
kind: Config
preferences: {}
users:
- name: gke_qwiklabs-gcp-01-4098456db096_us-central1-a_hello-world
  user:
    auth-provider:
      config:
        access-token: ya29.c.KpAB-wd7UfrKD5H6xwS8RZoyy7UToW9kLRy4zray1bVJNOlozejlZri36YrgoKZGkXgnXMfSMVyvXrGAM1lxJKjyh_3JjRKU3pCuNIhqpPRqKQe7mRFTBXa7XkLajV6Q5ZfHaDEEbfvvaHSUztgEVGJs_BwH9clKUhRWfLaQjLEgs5VUIDZCbtJ-teBGZBifrPDH
        cmd-args: config config-helper --format=json
        cmd-path: /usr/lib/google-cloud-sdk/bin/gcloud
        expiry: "2021-04-27T05:19:0

## Allow external traffic

By default, the pod is only accessible by its internal IP within the cluster. In order to make the hello-node container accessible from outside the Kubernetes virtual network, you have to expose the pod as a Kubernetes service.

You can expose the pod to the public internet with the `kubectl expose` command. The `--type="LoadBalancer"` flag is required for the creation of an externally accessible IP. This flag specifies that are using the load-balancer provided by the underlying infrastructure (in this case the Compute Engine load balancer). Note that you expose the deployment, and not the pod, directly. This will cause the resulting service to load balance traffic across all pods managed by the deployment (in this case only 1 pod, but you will add more replicas later).

**Exercise**

Expose the pod using `kubectl expose` and using the default load-balancer. 

In [29]:
%%bash
kubectl expose deployment hello-node --type="LoadBalancer" --port=8000

service/hello-node exposed


The Kubernetes master creates the load balancer and related Compute Engine forwarding rules, target pools, and firewall rules to make the service fully accessible from outside of Google Cloud.

To find the publicly-accessible IP address of the service, request kubectl to list all the cluster services.

In [33]:
!kubectl get services

NAME         TYPE           CLUSTER-IP     EXTERNAL-IP    PORT(S)          AGE
hello-node   LoadBalancer   10.7.246.232   34.68.245.49   8000:31425/TCP   66s
kubernetes   ClusterIP      10.7.240.1     <none>         443/TCP          5m32s


There are 2 IP addresses listed for your `hello-node` service, both serving port 8000. The `CLUSTER-IP` is the internal IP that is only visible inside your cloud virtual network; the `EXTERNAL-IP` is the external load-balanced IP.

You should now be able to reach the service by calling `curl http://<EXTERNAL_IP>:8000`.

In [34]:
!curl http://34.68.245.49:8000

No hello for no-world!
from Val


At this point you've gained several features from moving to containers and Kubernetes - you do not need to specify on which host to run your workload and you also benefit from service monitoring and restart. Now see what else can be gained from your new Kubernetes infrastructure.

## Scale up your service
One of the powerful features offered by Kubernetes is how easy it is to scale your application. Suppose you suddenly need more capacity. You can tell the replication controller to manage a new number of replicas for your pod: 
```bash
kubectl scale deployment hello-node --replicas=4
```

Scale your `hello-node` application to have 6 replicas. Then use `kubectl get` to request a description of the updated deployment and list all the pods:

In [31]:
!kubectl get deployment

NAME         READY   UP-TO-DATE   AVAILABLE   AGE
hello-node   1/1     1            1           4m40s


In [32]:
!kubectl get pods

NAME                         READY   STATUS    RESTARTS   AGE
hello-node-ff9ccfb86-gkrwh   1/1     Running   0          4m45s


**Exercise**

Use the `kubectl scale deployment` command to scale up the `hello-node` service to six machines.

In [35]:
%%bash
kubectl scale deployment hello-node --replicas=6

deployment.apps/hello-node scaled


In [36]:
!kubectl get deployment

NAME         READY   UP-TO-DATE   AVAILABLE   AGE
hello-node   6/6     6            6           5m33s


In [37]:
!kubectl get pods

NAME                         READY   STATUS    RESTARTS   AGE
hello-node-ff9ccfb86-42cph   1/1     Running   0          30s
hello-node-ff9ccfb86-52tzv   1/1     Running   0          30s
hello-node-ff9ccfb86-gkrwh   1/1     Running   0          5m34s
hello-node-ff9ccfb86-m4k6j   1/1     Running   0          30s
hello-node-ff9ccfb86-rcfh5   1/1     Running   0          30s
hello-node-ff9ccfb86-zwnt9   1/1     Running   0          30s


A declarative approach is being used here. Rather than starting or stopping new instances, you declare how many instances should be running at all times. Kubernetes reconciliation loops makes sure that reality matches what you requested and takes action if needed.

Here's a diagram summarizing the state of your Kubernetes cluster:

<img src='../assets/k8s_cluster.png' width="500">

## Roll out an upgrade to your service
At some point the application that you've deployed to production will require bug fixes or additional features. Kubernetes helps you deploy a new version to production without impacting your users.

First, modify the application by opening `server.js` so that the response is 
```bash
response.end("Hello Kubernetes World!");
```

Now you can build and publish a new container image to the registry with an incremented tag (`v2` in this case).

**Note**: Building and pushing this updated image should be quicker since caching is being taken advantage of.

**Exercise**

Build and push the updated image using the `hello_node.docker` file. Tag the updated image as `gcr.io/[project-id]/hello-node:v2`.

In [38]:
%%bash
docker build -f dockerfiles/hello_node.docker -t gcr.io/${PROJECT_ID}/hello-node:v2 .
docker push gcr.io/${PROJECT_ID}/hello-node:v2

Sending build context to Docker daemon  201.7kB
Step 1/4 : FROM node:6.9.2
 ---> faaadb4aaf9b
Step 2/4 : EXPOSE 8000
 ---> Using cache
 ---> ca24bc9011c0
Step 3/4 : COPY ./src/server.js .
 ---> d80c1502f796
Step 4/4 : CMD ["node", "server.js"]
 ---> Running in 7fecc4bcf48a
Removing intermediate container 7fecc4bcf48a
 ---> 8315a39f1329
Successfully built 8315a39f1329
Successfully tagged gcr.io/qwiklabs-gcp-01-4098456db096/hello-node:v2
The push refers to repository [gcr.io/qwiklabs-gcp-01-4098456db096/hello-node]
7abda04f61c2: Preparing
381c97ba7dc3: Preparing
604c78617f34: Preparing
fa18e5ffd316: Preparing
0a5e2b2ddeaa: Preparing
53c779688d06: Preparing
60a0858edcd5: Preparing
b6ca02dfe5e6: Preparing
53c779688d06: Waiting
60a0858edcd5: Waiting
604c78617f34: Layer already exists
fa18e5ffd316: Layer already exists
381c97ba7dc3: Layer already exists
0a5e2b2ddeaa: Layer already exists
53c779688d06: Layer already exists
b6ca02dfe5e6: Layer already exists
60a0858edcd5: Layer already exists


Kubernetes will smoothly update your replication controller to the new version of the application. In order to change the image label for your running container, you will edit the existing `hello-node` deployment and change the image from `gcr.io/PROJECT_ID/hello-node:v1` to `gcr.io/PROJECT_ID/hello-node:v2`.

To do this, use the `kubectl edit` command. It opens a text editor displaying the full deployment yaml configuration. It isn't necessary to understand the full yaml config right now, just understand that by updating the `spec.template.spec.containers.image` field in the config you are telling the deployment to update the pods with the new image.

Open a terminal and run the following command:

```bash 
kubectl edit deployment hello-node
```

Look for `Spec` > `containers` > `image` and change the version number to `v2`. This is the output you should see:

```bash
deployment.extensions/hello-node edited
```

New pods will be created with the new image and the old pods will be deleted. Run `kubectl get deployments` to confirm. 

**Note**: You may need to rerun the above command as it provisions machines.

In [48]:
!kubectl get deployments

NAME         READY   UP-TO-DATE   AVAILABLE   AGE
hello-node   4/4     4            4           13m


While this is happening, the users of your services shouldn't see any interruption. After a little while they'll start accessing the new version of your application. You can find more details on rolling updates in this documentation.

Hopefully with these deployment, scaling, and updated features, once you've set up your Kubernetes Engine cluster, you'll agree that Kubernetes will help you focus on the application rather than the infrastructure.

## Cleanup

Delete the cluster using `gcloud` to free up those resources. Use the `--quiet` flag if you are executing this in a notebook. Deleting the cluster can take a few minutes. 

In [49]:
!gcloud container clusters --quiet delete hello-world

Deleting cluster hello-world...done.                                           
Deleted [https://container.googleapis.com/v1/projects/qwiklabs-gcp-01-4098456db096/zones/us-central1-a/clusters/hello-world].


Copyright 2020 Google LLC Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

In [51]:
!kubectl get deployments

error: the server doesn't have a resource type "deployments"
